In [52]:

    
#!/usr/bin/env python
# coding: utf-8


import os
import sys
import re
import torch
from glob import glob
from openbabel import pybel
import numpy as np
import random
import pickle
import warnings
import requests
import os
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
from sklearn.model_selection import train_test_split
from dask.dataframe import from_pandas
from dask.multiprocessing import get

PYTORCH_ENABLE_MPS_FALLBACK=1
PYTORCH_ENABLE_SparseCPU_FALLBACK=1
warnings.filterwarnings("ignore")

from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1024,
                       processes=12,
                       memory="250GB",
                      # account="hmslati",
                      # walltime="01:00:00",
                       queue="gpu-bigmem")

# PLEASE READ -> 45次实验分别进行10倍交叉验证，取平均

#Converts the protein-ligand complexes into 4D tensor. 
class Feature_extractor():
    def __init__(self):
        self.atom_codes = {}
        #'others' includs metal atoms and B atom. There are no B atoms on training and test sets. 
        # 55 to 63 will be reserved to PLIF features as follows:
        # 55: hydrophobic
        # 56: hbond
        # 57: waterbridge
        # 58: saltbridge
        # 59: pistacking
        # 60: pication
        # 61: halogen
        # 62: metal
        # 63: Distances 
        
        # others = ([3,4,5,11,12,13]+list(range(19,32))+list(range(37,51))+list(range(55,84)))
        plif_specs=list(range(55,64))
        #C and N atoms can be hybridized in three ways and S atom can be hybridized in two ways here. 
        #Hydrogen atom is also considered for feature extraction. I think phosphor atom has 3 or 5 as hyb states but 
        # in biological system its usually the same recurrent phosphate even in most small molecules so safe to assume one
        # hybridization state for this purpose. 
        atom_types = [1,(6,1),(6,2),(6,3),(7,1),(7,2),(7,3),8,15,(16,2),(16,3),
                      34,9,17,35,53,11,12,13,14,5,19,20,25,29,28,30,33]+plif_specs
      
        for i, j in enumerate(atom_types):
            if type(j) is list:
                for k in j:
                    self.atom_codes[k] = i
                
            else:
                self.atom_codes[j] = i              
        
        self.sum_atom_types = len(atom_types)
        
    #Onehot encoding of each atom. The atoms in protein or ligand are treated separately.
    def encode(self, atomic_num, orig_coords, plifs, molprotein):
        encoding = np.zeros(self.sum_atom_types*2)
        if molprotein == 1:
            encoding[self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.atom_codes[55]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.atom_codes[56]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.atom_codes[57]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.atom_codes[58]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.atom_codes[59]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.atom_codes[60]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.atom_codes[61]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.atom_codes[62]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.atom_codes[63]] = plifs[coord][1]
                    

                
        else:
            encoding[self.sum_atom_types+self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.sum_atom_types+self.atom_codes[55]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[56]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[57]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[58]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[59]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[60]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[61]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[62]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.sum_atom_types+self.atom_codes[63]] = plifs[coord][1]

        
        return encoding
    
    #Get atom coords and atom features from the complexes.   
    def get_features(self, molecule, plifs, molprotein):
        coords = []
        features = []
            
        for atom in molecule:
            coords.append(atom.coords)
            if atom.atomicnum in [6,7,16]:
                atomicnum = (atom.atomicnum,atom.hyb)
                features.append(self.encode(atomicnum,atom.coords,plifs,molprotein))
            else:
                features.append(self.encode(atom.atomicnum,atom.coords,plifs,molprotein))
        
        coords = np.array(coords, dtype=np.float32)
        features = np.array(features, dtype=np.float32)

        return coords, features
     
    #Define the rotation matrixs of 3D stuctures.
    def rotation_matrix(self, t, roller):
        if roller==0:
            return np.array([[1,0,0],[0,np.cos(t),np.sin(t)],[0,-np.sin(t),np.cos(t)]])
        elif roller==1:
            return np.array([[np.cos(t),0,-np.sin(t)],[0,1,0],[np.sin(t),0,np.cos(t)]])
        elif roller==2:
            return np.array([[np.cos(t),np.sin(t),0],[-np.sin(t),np.cos(t),0],[0,0,1]])

    #Generate 3d grid or 4d tensor. Each grid represents a voxel. Each voxel represents the atom in it by onehot encoding of atomic type.
    #Each complex in train set is rotated 9 times for data amplification.
    #The complexes in core set are not rotated. 
    #The default resolution is 20*20*20.
    def grid(self,grid, coords, features, frag_idx, resolution=1.0, max_dist=10.0,  rotation_bool=True, max_frag=10, rotations=9):
        assert coords.shape[1] == 3
        assert coords.shape[0] == features.shape[0]  

        slider=frag_idx*20

        x=y=z=np.array(range(-10,10),dtype=np.float32)+0.5
        u=0
        for i in range(len(coords)):
            coord=coords[i]
            # add/subtract 10 from the center
            tmpx=abs(coord[0]-x)
            tmpy=abs(coord[1]-y)
            tmpz=abs(coord[2]-z)

            if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                u+=1
                # get the position of the closest point to coordinate which is found inside the grid
                # append the features unto that slice

                grid[0,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += torch.tensor(features[i])
                
        if rotation_bool:
            for rotation_idx in range(rotations):
                theta = random.uniform(np.pi/18,np.pi/2)
                roller = random.randrange(3)
                coords = np.dot(coords, self.rotation_matrix(theta,roller))
                for i in range(len(coords)):
                    coord=coords[i]
                    tmpx=abs(coord[0]-x)
                    tmpy=abs(coord[1]-y)
                    tmpz=abs(coord[2]-z)
                    if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                        grid[rotation_idx+1,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += features[i]

        return grid
    
class PLIF:
    def __init__(self, PDB: str, MOL_SPLIT_START: int = 70, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super(PLIF,self).__init__()
        
        self.MOL_SPLIT_START=MOL_SPLIT_START
        self.pdb=PDB
        self.records=['ATOM']
        self.values=['HOH','CL','MG','ZN','MN','CA']
        self.ions=['CL','MG','ZN','MN','CA']
        self.interaction_slices={"hydrophobic":[0,1,6,7,8,9,10],
            "hbond":[0,1,7,11,13,15,16],
            "waterbridge":[0,1,[6,7],11,13,16,17],
            "saltbridge":[0,1,7,10,3,11,12],
            "pistacking":[0,1,7,11,6,12,13],
            "pication":[0,1,7,11,3,12,13],
            "halogen":[0,1,7,10,12,14,15],
            "metal":[0,1,11,8,6,17,16]} 

        self.column_names = ['RESNR', 'RESTYPE', 'DIST', 'LIG_IDX','PROT_IDX','FRAGMENT_ATOMS_COORDS', 'AA_COORDS']
        self.path = os.getcwd()


    def okToBreak(self, bond):
        """
        Here we apply a bunch of rules to judge if the bond is OK to break.

        Parameters
        ----------
        bond :
            RDkit MOL object

        Returns
        -------
        Boolean :
            OK or not to break.
        """
        # See if the bond is in Ring (don't break that)
        if bond.IsInRing():
            return False
        # We OK only single bonds to break
        if bond.GetBondType() != Chem.rdchem.BondType.SINGLE:
            return False

        # Get the beginning atom of the bond
        begin_atom = bond.GetBeginAtom()
        # Get the ending atom of the bond
        end_atom = bond.GetEndAtom()
        # What kind of neighbors does these end and begenning atoms have? We need a family of no less than 5!
        neighbor_end=list(end_atom.GetNeighbors())
        neighbor_begin=list(begin_atom.GetNeighbors())
        if (len(neighbor_end) + len(neighbor_begin)) <5:
            return False
        #for atm in neighbor_end:
            #print(atm.GetAtomicNum())
        #print(begin_atom.GetAtomicNum(), end_atom.GetAtomicNum(), MOL_SPLIT_START)
        
        # Now check if end or begenning atoms are in ring (we dont wanna bother those)
        if not(begin_atom.IsInRing() or end_atom.IsInRing()):
            return False
        elif begin_atom.GetAtomicNum() >= self.MOL_SPLIT_START or \
                end_atom.GetAtomicNum() >= self.MOL_SPLIT_START:
            return False
        elif end_atom.GetAtomicNum() == 1:
            return False
        else:
            return True

    def undo_id_label (self, frag, split_id):
        # I am trying to restore Hydrogens where the break happened
        for i, atom in enumerate(frag.GetAtoms()):
            if atom.GetAtomicNum() >= split_id:
                atom.SetAtomicNum(1)

        return frag

    # Divide a molecule into fragments
    def split_molecule(self, mol, pdb):

        split_id = self.MOL_SPLIT_START

        res = []
        res_no_id=[]

        to_check = [mol]
        while len(to_check) > 0:
            ms = self.spf(to_check.pop(), split_id)
            if len(ms) == 1:
                res += ms
            else:
                to_check += ms
                split_id += 1
        for frag in res:
            res_no_id.append(self.undo_id_label(frag, self.MOL_SPLIT_START))

        res_pdb_frags=[]

        for idx, frag in enumerate(res_no_id):
            w = Chem.PDBWriter(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
            w.write(frag)
            w.close()
            
            unwanted_entries= ['CONECT', 'END']            
            with open(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as oldfile, open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as newfile:
                for line in oldfile:
                    if not any(unwanted_entry in line for unwanted_entry in unwanted_entries):
                        newfile.write(line)

                    
            data = data2 = ""

            # Reading data from file1
            with open(f"ATOM_{pdb}.pdb") as fp:
                data = fp.read()

            # Reading data from file2

            with open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as fp:
                data2 = fp.read()
            
            # Merging 2 files
            # To add the data of file2
            # from next line
            #data += "\n"
            data += data2
            
            with open(f"HOH_{pdb}.pdb") as fp:
                data3 = fp.read()
            data += data3

            with open (f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as fp:
                fp.write(data)
            res_pdb_frags.append(f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        return res_pdb_frags #create_chain(res)


    # Function for doing all the nitty gritty splitting work.
    # loops over bonds until bonds get exhausted or bonds are ok to break, whichever comes first. If ok to break, then each
    # fragment needs to be checked individually again through the loop
    def spf(self, mol, split_id):

        bonds = mol.GetBonds()
        for i in range(len(bonds)):
            if self.okToBreak(bonds[i]):
                mol = Chem.FragmentOnBonds(mol, [i])
                # Dummy atoms are always added last
                n_at = mol.GetNumAtoms()
                print('Split ID', split_id)
                mol.GetAtomWithIdx(n_at-1).SetAtomicNum(split_id)
                mol.GetAtomWithIdx(n_at-2).SetAtomicNum(split_id)
                return Chem.rdmolops.GetMolFrags(mol, asMols=True)

        # If the molecule could not been split, return original molecule
        return [mol]
    #get_fragments(fragment_mols)

    def retreive_plip_interactions(self, pdb_file):
        """
        Retreives the interactions from PLIP.

        Parameters
        ----------
        pdb_file :
            The PDB file of the complex. 

        Returns
        -------
        dict :
            A dictionary of the binding sites and the interactions.
        """
        protlig = PDBComplex()   #instantiate the loader from PLIP
        protlig.load_pdb(pdb_file)   # load the pdb file
        for ligand in protlig.ligands:
            protlig.characterize_complex(ligand)   # find ligands and analyze interactions
        sites = {}
        # loop over binding sites
        for key, site in sorted(protlig.interaction_sets.items()):
            binding_site = BindingSiteReport(site)   # collect data about interactions
            # tuples of *_features and *_info will be converted to pandas DataFrame
            keys = (
                "hydrophobic",
                "hbond",
                "waterbridge",
                "saltbridge",
                "pistacking",
                "pication",
                "halogen",
                "metal"
            )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with 
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')
        # 2. information for each of these features, e.g. for hydrophobic
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')

            interactions = {
                k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
                for k in keys
            }
            sites[key] = interactions
        return sites

    def get_coords_prot(self, RESNR):
        ppdb = PandasPdb()
        ppdb.read_pdb(f"{self.pdb.split('.')[0]}_protein.pdb")
        only_protein=ppdb.df['ATOM']
        resnr_coords=[]
        for i in RESNR:
            resnr_coords.append(list(only_protein[only_protein['atom_number']==int(i)][['x_coord', 'y_coord', 'z_coord']].values[0]))
        return resnr_coords
    
    def interaction_df(self, split):

        all_interactions_df = pd.DataFrame()


        # We create the dictionary for the complex of interest:
        for idx, s in enumerate(split):

            pdb_id=s.split('.')[0]
            raw=pdb_id.split('_')[1]
            idx_frag=int(pdb_id.split('_')[2])
            interactions_by_site = self.retreive_plip_interactions(f"{pdb_id}.pdb")

            # Let’s see how many binding sites are detected:

    #         print(
    #             f"Number of binding sites detected in {pdb_id} : "
    #             f"{len(interactions_by_site)}\n"
    #             f"with {interactions_by_site.keys()}"
    #         )
            # In this case, the first binding site containing ligand 03P will be further investigated.
            index_of_selected_site = 0
            selected_site = list(interactions_by_site.keys())[index_of_selected_site]
            #print(selected_site)


            valid_types = [
                    "hydrophobic",
                    "hbond",
                    "waterbridge",
                    "saltbridge",
                    "pistacking",
                    "pication",
                    "halogen",
                    "metal",
                ]

            for _type in valid_types:
                output_df=self.create_df_from_binding_site(raw, interactions_by_site[selected_site], idx+self.MOL_SPLIT_START, selected_site,
                                                      interactions_by_site,
                                                      interaction_type=_type)
                all_interactions_df=all_interactions_df.append(output_df)
        all_interactions_df = all_interactions_df[all_interactions_df['RESNR'].notna()]
        all_interactions_df.to_csv(f"{self.path}/results_plifs/{raw}_plifs_and_properties.csv", index=False)
        return all_interactions_df


    # We can construct a pandas.DataFrame for a binding site and particular interaction type.

    def create_df_from_binding_site(self, raw, selected_site_interactions, fragment_idx, selected_site, 
                                    interactions_by_site, interaction_type="hbond"):
        """
        Creates a data frame from a binding site and interaction type.

        Parameters
        ----------
        selected_site_interactions : dict
            Precalculated interactions from PLIP for the selected site
        interaction_type : str
            The interaction type of interest (default set to hydrogen bonding).

        Returns
        -------
        pd.DataFrame :
            DataFrame with information retreived from PLIP.
        """
        # check if interaction type is valid:
        valid_types = [
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        ]


        if interaction_type not in valid_types:
            print("!!! Wrong interaction type specified. Hbond is chosen by default !!! \n")
            interaction_type = "hbond"

        def interaction_values(n):
            try:
                interactions=interactions_by_site[selected_site][interaction_type]
                if type(n) is list:
                    return [interactions[1:][x][i] for x in 
                        range(len(interactions[1:])) for i in n]
                else:
                    return [interactions[1:][x][n] for x in 
                        range(len(interactions[1:]))]
            except Exception:
                return None
            
        if interactions_by_site[selected_site][interaction_type][1:]:
            #print(list(map(interaction_values, self.interaction_slices[interaction_type])), self.column_names)
            selected_feats=list(map(interaction_values, self.interaction_slices[interaction_type]))
            #print(selected_feats)
            try: 
                if int(selected_feats[4])>int(selected_feats[3]):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3]  
            except: 
                if int(any(selected_feats[4]))>int(any(selected_feats[3])):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3] 
            df = pd.DataFrame(
                # data is stored AFTER the columns names
                [selected_feats],
                # column names are always the first element - we skipped that in the above - we are gonna use that for naming the df
                columns = self.column_names
            )

            df["INTERACTION_TYPE"]=interaction_type
            
            try:
                checked_coords=self.get_coords_prot(selected_feats[4][0].split(',') if ',' in selected_feats[4][0] \
                                                                   else selected_feats[4])
            except:
                checked_coords=selected_feats[6]
                
            df["AA_COORDS"]=[checked_coords]
                #[self.get_coords_prot(selected_feats[4].split(','))]
            df["FRAGMENT_ATOMS_COORDS"]=[selected_feats[5]]
                            #[self.get_coords_lig(selected_feats[3].split(','))]    
            df['FRAGMENT_ID']=fragment_idx

            # ideally we would like to exclude waters from further processing. Threrfore let us reduce any waterbridge 
            # interaction to the eucladean distance in order to omit water
            
            if interaction_type == "waterbridge":
                df['DIST']=[[np.linalg.norm(x) for x in df['DIST'].to_numpy()]]
                
            # also deal with one distance value and two coords, this is common in saltbridge interactions:
            if len(checked_coords) == len(selected_feats[2])*2:
                df['DIST']=[selected_feats[2] + selected_feats[2]]
                
        else:

            df= pd.DataFrame({'RESNR':[None], 'RESTYPE':[None], 'DIST':[None], 'LIG_IDX':[None],'PROT_IDX':[None],
                        'INTERACTION_TYPE':[interaction_type], "AA_COORDS": [None], "FRAGMENT_ATOMS_COORDS":[None],
                              'FRAGMENT_ID':[str(fragment_idx)]})



        return df
    
    def pdb_2_sdf(self, pdb):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("pdb", "sdf")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

        mol.AddHydrogens()


        obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
        return f"{pdb.split('.')[0]}.sdf"
    
    def sdf_2_pdb(self, sdf):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("sdf", "pdb")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

        mol.AddHydrogens()
        obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
        return f"HETATM_{sdf.split('.')[0]}.pdb"

    def save_bpdb(self, pdb,ppdb, record):  
        ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                    records=[record],
                    gz=False, 
                    append_newline=True)

    def get_HOH_pdb(self, pdb):
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb) 
        ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(self.values)]
        ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=['HETATM'],
                gz=False, 
                append_newline=True)

    def keep_relevant_hetatm(self, pdb):
        raw=str(self.pdb).split('.')[0]
        with open(pdb) as f1, open(f"ATOM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if 'ATOM' in line:
                    f2.write(line)
        with open(f'{raw}_ligand.pdb') as f1, open(f"HETATM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if ('HETATM' in line) and not any(ion in line for ion in self.ions):
                    f2.write(line)
        try: 
            self.get_HOH_pdb(pdb)
        except:
            with open(pdb) as f1, open(f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
                for line in f1:
                    if ('HETATM' in line) and any(ion in line for ion in self.ions):
                        f2.write(line)
        return
    
    
    def fragment_and_plif(self):
        path = os.getcwd()
        if not os.path.exists('results_plifs'):
            os.mkdir(f'{path}/results_plifs')

        raw=str(self.pdb).split('.')[0]
        self.sdf_2_pdb(f'{raw}_ligand.sdf')
        self.keep_relevant_hetatm(f'{raw}_protein.pdb')
        fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
        fragment_mols_alt = Chem.MolFromMol2File(f'{raw}_ligand.mol2', sanitize=True, removeHs=True)
        content = open(f'{raw}_ligand.pdb').read()
        hets=re.findall("^HETATM (.*)", content, re.M)
        if len(hets)<5:
            # Read in the file
            with open(f'{raw}_ligand.pdb', 'r') as file :
                filedata = file.read()

            # Replace the target string
            filedata = filedata.replace('ATOM  ', 'HETATM')

            # Write the file out again
            with open(f'{raw}_ligand.pdb', 'w') as file:
                file.write(filedata)
        
        try: 
            fragment_mols = Chem.RemoveHs(fragment_mols[0])
            output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
            
        except:  
            try: 
                output_df = self.interaction_df(self.split_molecule( Chem.MolFromMol2File(fragment_mols_alt)))
            except:
                try: 
                    fragment_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
                    output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                except:
                    try:
                        fragment_mols = AllChem.MolFromPDBFile(f'HETATM_{raw}.pdb')
                        output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                    except:
                        raise Exception(f"Sorry, the pdb id: {raw} needs chekcing")
#                         output_df= pd.DataFrame({'FRAGMENT_ID':[None], 'AA_COORDS':[None], 
#                                                  'FRAGMENT_ATOMS_COORDS':[None], 
#                                      'INTERACTION_TYPE':[None],'DIST':[None]})
        os.chdir(f'{path}')

        return output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST'].agg(list)


def kd_equalizer (value):

    if 'mM' in value.split('=')[1]:
        return float(value.split('m')[0].split('=')[1]) / 1000
    elif 'uM' in value.split('=')[1]:
        return float(value.split('u')[0].split('=')[1]) / 1000000
    elif 'nM' in value.split('=')[1]:
        return float(value.split('n')[0].split('=')[1]) / 1000000000
    elif 'pM' in value.split('=')[1]:
        return float(value.split('p')[0].split('=')[1]) / 1000000000000
    elif 'fM' in value.split('=')[1]:
        return float(value.split('f')[0].split('=')[1]) / 1000000000000000


if __name__ == "__main__":
    
    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/')
             
    train_grids=None
    test_grids=None
    rotations=9
    full_batch=10
    features_shape=74
    
    Feature = Feature_extractor()
    
    p_directory = os.getcwd()
    
    general=pd.read_csv('INDEX_general_PL_data.2020', sep=',')
    refined=pd.read_csv('INDEX_refined_data.2020', sep=',')
    
    general=general[general["Kd/Ki"].str.contains('IC|EC|>|<')==False]
    refined=refined[refined["Kd/Ki"].str.contains('IC|EC|>|<')==False]

    general["Kd/Ki"] = general["Kd/Ki"].str.replace('~','=')
    refined["Kd/Ki"] = refined["Kd/Ki"].str.replace('~','=')


    general['Kd/Ki']=general['Kd/Ki'].apply(lambda x: kd_equalizer(x))
    refined['Kd/Ki']=refined['Kd/Ki'].apply(lambda x: kd_equalizer(x))
    
    merged_PDBBind=general.append(refined) \
                                .sample(frac=1) \
                                .sample(frac=1) \
                                .reset_index(drop=True) \
                                .drop_duplicates(subset='PDB_code', keep="first") 

    merged_PDBBind[merged_PDBBind['PDB_code'].str.contains("3bho")==False]
    
    train_df, test_df = train_test_split(merged_PDBBind, test_size=0.1)
    
    merged_PDBBind.to_csv('merged_PDBBind.csv', index=False)
    train_df.to_csv('train_df.csv', index=False)
    test_df.to_csv('test_df.csv', index=False)
    
    # First training grids: 
    train_label=[]
    
    for _ , row in train_df.iterrows():
        
        pdb_id = row['PDB_code']
        print('pdb_id', pdb_id)
        
        os.chdir(f'general_refined_set/{pdb_id}')
        
        raw=pdb_id
        path = os.getcwd()
        fileList = []
        fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
        for filePath in fileList:
            try:
                os.remove(filePath)
            except:
                print("Error while deleting file : ", filePath)
                
        df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
        
        train_label.extend([row['Kd/Ki']]*10)
        
        single_pdb_frags = []
        for idx, row in df_plifSpecs.iterrows():
    
            temp_plifs_prot={}
            temp_plifs_frag={}
            ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
            for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                               row['FRAGMENT_ATOMS_COORDS'],
                                                                                               row['INTERACTION_TYPE'],
                                                                                               row['DIST']):
                # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
                # for two amino acids or ligand atoms! Encoding by atom is crazy fun
                for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                                 frag_lig_atm_coord_list):
                    temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                    temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


            pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
            ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
            single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  
            
        grid=torch.zeros((full_batch,400,400,400,features_shape))
        
        for idx, mols in enumerate(single_pdb_frags):
            print(len(single_pdb_frags), idx)
#             threads = multiprocessing.Pool(len(single_pdb_frags))
#             threads.map(func, arg_list)
            
            coords1, features1 = Feature.get_features(mols[0],mols[2],1)
            coords2, features2 = Feature.get_features(mols[1],mols[3],0)

            # get the center point of protein
            center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
            coords=np.concatenate([coords1,coords2],axis = 0)
            features=np.concatenate([features1,features2],axis = 0)
            assert len(coords) == len(features)
            # zero the coordinates 
            coords = coords-center
            grid=Feature.grid(grid,coords,features,idx, rotation_bool=True)
        
        if train_grids is None:
            train_grids = grid.to_sparse()
        else:
            train_grids = torch.cat((train_grids,grid.to_sparse()), 0)
        print(train_grids.shape)
        
        raw=pdb_id
        path = os.getcwd()
        fileList = []
        fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
        for filePath in fileList:
            try:
                os.remove(filePath)
            except:
                print("Error while deleting file : ", filePath)
                
        os.chdir(p_directory)
        
        print("Memory utilised (bytes): ", sys.getsizeof(train_grids))
    with open('train_grids.pkl','wb') as f:
        pickle.dump(train_grids, f)

    # Second testing grids: 
    test_label=[]
    
    for _ , row in test_df.iterrows():
        
        pdb_id = row['PDB_code']
        print('pdb_id', pdb_id)
        
        os.chdir(f'general_refined_set/{pdb_id}')
        
        raw=pdb_id
        path = os.getcwd()
        fileList = []
        fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
        for filePath in fileList:
            try:
                os.remove(filePath)
            except:
                print("Error while deleting file : ", filePath)
                
        df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
        
        test_label.extend([row['Kd/Ki']])
        
        single_pdb_frags = []
        print(len(single_pdb_frags))
        for idx, row in df_plifSpecs.iterrows():
    
            temp_plifs_prot={}
            temp_plifs_frag={}
            ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
            for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                               row['FRAGMENT_ATOMS_COORDS'],
                                                                                               row['INTERACTION_TYPE'],
                                                                                               row['DIST']):
                # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
                # for two amino acids or ligand atoms! Encoding by atom is crazy fun
                for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                                 frag_lig_atm_coord_list):
                    temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                    temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


            pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
            ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
            single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  
            
        grid=torch.zeros((1,400,400,400,features_shape))

        for idx, mols in enumerate(single_pdb_frags):
            coords1, features1 = Feature.get_features(mols[0],mols[2],1)
            coords2, features2 = Feature.get_features(mols[1],mols[3],0)

            # get the center point of protein
            center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
            coords=np.concatenate([coords1,coords2],axis = 0)
            features=np.concatenate([features1,features2],axis = 0)
            assert len(coords) == len(features)
            # zero the coordinates 
            coords = coords-center
            grid=Feature.grid(grid,coords,features,idx, rotation_bool=False)
        
        if test_grids is None:
            test_grids = grid.to_sparse()
        else:
            test_grids = torch.cat((test_grids,grid.to_sparse()), 0)
        print(test_grids.shape)
        
        raw=pdb_id
        path = os.getcwd()
        fileList = []
        fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
        for filePath in fileList:
            try:
                os.remove(filePath)
            except:
                print("Error while deleting file : ", filePath)
                
        os.chdir(p_directory)
        
    
    with open('test_grids.pkl','wb') as f:
        pickle.dump(test_grids, f)
        
    


pdb_id 1df8
Split ID 70
2 0
2 1


KeyboardInterrupt: 

In [31]:

    
#!/usr/bin/env python
# coding: utf-8


import os
import sys
import re
import torch
from glob import glob
from openbabel import pybel
import numpy as np
import random
import pickle
import warnings
import requests
import os
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
from sklearn.model_selection import train_test_split
from dask.dataframe import from_pandas
from dask.multiprocessing import get
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False)


PYTORCH_ENABLE_MPS_FALLBACK=1
PYTORCH_ENABLE_SparseCPU_FALLBACK=1
warnings.filterwarnings("ignore")

from dask_jobqueue import SLURMCluster

train_grids=None
test_grids=None
rotations=9
full_batch=10
features_shape=74
# First training grids: 
train_label=[]
# Second testing grids: 
test_label=[]
    
# cluster = SLURMCluster(cores=1024,
#                        processes=12,
#                        memory="250GB",
#                       # account="hmslati",
#                       # walltime="01:00:00",
#                        queue="gpu-bigmem")

# PLEASE READ -> 45次实验分别进行10倍交叉验证，取平均

#Converts the protein-ligand complexes into 4D tensor. 
class Feature_extractor():
    def __init__(self):
        self.atom_codes = {}
        #'others' includs metal atoms and B atom. There are no B atoms on training and test sets. 
        # 55 to 63 will be reserved to PLIF features as follows:
        # 55: hydrophobic
        # 56: hbond
        # 57: waterbridge
        # 58: saltbridge
        # 59: pistacking
        # 60: pication
        # 61: halogen
        # 62: metal
        # 63: Distances 
        
        # others = ([3,4,5,11,12,13]+list(range(19,32))+list(range(37,51))+list(range(55,84)))
        plif_specs=list(range(55,64))
        #C and N atoms can be hybridized in three ways and S atom can be hybridized in two ways here. 
        #Hydrogen atom is also considered for feature extraction. I think phosphor atom has 3 or 5 as hyb states but 
        # in biological system its usually the same recurrent phosphate even in most small molecules so safe to assume one
        # hybridization state for this purpose. 
        atom_types = [1,(6,1),(6,2),(6,3),(7,1),(7,2),(7,3),8,15,(16,2),(16,3),
                      34,9,17,35,53,11,12,13,14,5,19,20,25,29,28,30,33]+plif_specs
      
        for i, j in enumerate(atom_types):
            if type(j) is list:
                for k in j:
                    self.atom_codes[k] = i
                
            else:
                self.atom_codes[j] = i              
        
        self.sum_atom_types = len(atom_types)
        
    #Onehot encoding of each atom. The atoms in protein or ligand are treated separately.
    def encode(self, atomic_num, orig_coords, plifs, molprotein):
        encoding = np.zeros(self.sum_atom_types*2)
        if molprotein == 1:
            encoding[self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.atom_codes[55]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.atom_codes[56]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.atom_codes[57]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.atom_codes[58]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.atom_codes[59]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.atom_codes[60]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.atom_codes[61]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.atom_codes[62]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.atom_codes[63]] = plifs[coord][1]
                    

                
        else:
            encoding[self.sum_atom_types+self.atom_codes[atomic_num]] = 1.0
            for coord, plif_feats in plifs.items():
                if [round(item) for item in coord] == [round(item) for item in orig_coords]:
                    encoding[self.sum_atom_types+self.atom_codes[55]] = 1.0 if plifs[coord][0] == 'hydrophobic' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[56]] = 1.0 if plifs[coord][0] == 'hbond' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[57]] = 1.0 if plifs[coord][0] == 'waterbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[58]] = 1.0 if plifs[coord][0] == 'saltbridge' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[59]] = 1.0 if plifs[coord][0] == 'pistacking' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[60]] = 1.0 if plifs[coord][0] == 'pication' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[61]] = 1.0 if plifs[coord][0] == 'halogen' \
                    else 0.0
                    encoding[self.sum_atom_types+self.atom_codes[62]] = 1.0 if plifs[coord][0] == 'metal' \
                    else 0.0
                    
                    #distance
                    encoding[self.sum_atom_types+self.atom_codes[63]] = plifs[coord][1]

        
        return encoding
    
    #Get atom coords and atom features from the complexes.   
    def get_features(self, molecule, plifs, molprotein):
        coords = []
        features = []
            
        for atom in molecule:
            coords.append(atom.coords)
            if atom.atomicnum in [6,7,16]:
                atomicnum = (atom.atomicnum,atom.hyb)
                features.append(self.encode(atomicnum,atom.coords,plifs,molprotein))
            else:
                features.append(self.encode(atom.atomicnum,atom.coords,plifs,molprotein))
        
        coords = np.array(coords, dtype=np.float32)
        features = np.array(features, dtype=np.float32)

        return coords, features
     
    #Define the rotation matrixs of 3D stuctures.
    def rotation_matrix(self, t, roller):
        if roller==0:
            return np.array([[1,0,0],[0,np.cos(t),np.sin(t)],[0,-np.sin(t),np.cos(t)]])
        elif roller==1:
            return np.array([[np.cos(t),0,-np.sin(t)],[0,1,0],[np.sin(t),0,np.cos(t)]])
        elif roller==2:
            return np.array([[np.cos(t),np.sin(t),0],[-np.sin(t),np.cos(t),0],[0,0,1]])

    #Generate 3d grid or 4d tensor. Each grid represents a voxel. Each voxel represents the atom in it by onehot encoding of atomic type.
    #Each complex in train set is rotated 9 times for data amplification.
    #The complexes in core set are not rotated. 
    #The default resolution is 20*20*20.
    def grid(self,grid, coords, features, frag_idx, resolution=1.0, max_dist=10.0,  rotation_bool=True, max_frag=10, rotations=9):
        assert coords.shape[1] == 3
        assert coords.shape[0] == features.shape[0]  

        slider=frag_idx*20

        x=y=z=np.array(range(-10,10),dtype=np.float32)+0.5
        u=0
        for i in range(len(coords)):
            coord=coords[i]
            # add/subtract 10 from the center
            tmpx=abs(coord[0]-x)
            tmpy=abs(coord[1]-y)
            tmpz=abs(coord[2]-z)

            if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                u+=1
                # get the position of the closest point to coordinate which is found inside the grid
                # append the features unto that slice

                grid[0,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += torch.tensor(features[i])
                
        if rotation_bool:
            for rotation_idx in range(rotations):
                theta = random.uniform(np.pi/18,np.pi/2)
                roller = random.randrange(3)
                coords = np.dot(coords, self.rotation_matrix(theta,roller))
                for i in range(len(coords)):
                    coord=coords[i]
                    tmpx=abs(coord[0]-x)
                    tmpy=abs(coord[1]-y)
                    tmpz=abs(coord[2]-z)
                    if np.max(tmpx)<=19.5 and np.max(tmpy)<=19.5 and np.max(tmpz) <=19.5:
                        grid[rotation_idx+1,slider+np.argmin(tmpx),slider+np.argmin(tmpy),slider+np.argmin(tmpz)] += features[i]

        return grid
    
class PLIF:
    def __init__(self, PDB: str, MOL_SPLIT_START: int = 70, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super(PLIF,self).__init__()
        
        self.MOL_SPLIT_START=MOL_SPLIT_START
        self.pdb=PDB
        self.records=['ATOM']
        self.values=['HOH','CL','MG','ZN','MN','CA']
        self.ions=['CL','MG','ZN','MN','CA']
        self.interaction_slices={"hydrophobic":[0,1,6,7,8,9,10],
            "hbond":[0,1,7,11,13,15,16],
            "waterbridge":[0,1,[6,7],11,13,16,17],
            "saltbridge":[0,1,7,10,3,11,12],
            "pistacking":[0,1,7,11,6,12,13],
            "pication":[0,1,7,11,3,12,13],
            "halogen":[0,1,7,10,12,14,15],
            "metal":[0,1,11,8,6,17,16]} 

        self.column_names = ['RESNR', 'RESTYPE', 'DIST', 'LIG_IDX','PROT_IDX','FRAGMENT_ATOMS_COORDS', 'AA_COORDS']
        self.path = os.getcwd()


    def okToBreak(self, bond):
        """
        Here we apply a bunch of rules to judge if the bond is OK to break.

        Parameters
        ----------
        bond :
            RDkit MOL object

        Returns
        -------
        Boolean :
            OK or not to break.
        """
        # See if the bond is in Ring (don't break that)
        if bond.IsInRing():
            return False
        # We OK only single bonds to break
        if bond.GetBondType() != Chem.rdchem.BondType.SINGLE:
            return False

        # Get the beginning atom of the bond
        begin_atom = bond.GetBeginAtom()
        # Get the ending atom of the bond
        end_atom = bond.GetEndAtom()
        # What kind of neighbors does these end and begenning atoms have? We need a family of no less than 5!
        neighbor_end=list(end_atom.GetNeighbors())
        neighbor_begin=list(begin_atom.GetNeighbors())
        if (len(neighbor_end) + len(neighbor_begin)) <5:
            return False
        #for atm in neighbor_end:
            #print(atm.GetAtomicNum())
        #print(begin_atom.GetAtomicNum(), end_atom.GetAtomicNum(), MOL_SPLIT_START)
        
        # Now check if end or begenning atoms are in ring (we dont wanna bother those)
        if not(begin_atom.IsInRing() or end_atom.IsInRing()):
            return False
        elif begin_atom.GetAtomicNum() >= self.MOL_SPLIT_START or \
                end_atom.GetAtomicNum() >= self.MOL_SPLIT_START:
            return False
        elif end_atom.GetAtomicNum() == 1:
            return False
        else:
            return True

    def undo_id_label (self, frag, split_id):
        # I am trying to restore Hydrogens where the break happened
        for i, atom in enumerate(frag.GetAtoms()):
            if atom.GetAtomicNum() >= split_id:
                atom.SetAtomicNum(1)

        return frag

    # Divide a molecule into fragments
    def split_molecule(self, mol, pdb):

        split_id = self.MOL_SPLIT_START

        res = []
        res_no_id=[]

        to_check = [mol]
        while len(to_check) > 0:
            ms = self.spf(to_check.pop(), split_id)
            if len(ms) == 1:
                res += ms
            else:
                to_check += ms
                split_id += 1
        for frag in res:
            res_no_id.append(self.undo_id_label(frag, self.MOL_SPLIT_START))

        res_pdb_frags=[]

        for idx, frag in enumerate(res_no_id):
            w = Chem.PDBWriter(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
            w.write(frag)
            w.close()
            
            unwanted_entries= ['CONECT', 'END']            
            with open(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as oldfile, open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as newfile:
                for line in oldfile:
                    if not any(unwanted_entry in line for unwanted_entry in unwanted_entries):
                        newfile.write(line)

                    
            data = data2 = ""

            # Reading data from file1
            with open(f"ATOM_{pdb}.pdb") as fp:
                data = fp.read()

            # Reading data from file2

            with open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as fp:
                data2 = fp.read()
            
            # Merging 2 files
            # To add the data of file2
            # from next line
            #data += "\n"
            data += data2
            
            with open(f"HOH_{pdb}.pdb") as fp:
                data3 = fp.read()
            data += data3

            with open (f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as fp:
                fp.write(data)
            res_pdb_frags.append(f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        return res_pdb_frags #create_chain(res)


    # Function for doing all the nitty gritty splitting work.
    # loops over bonds until bonds get exhausted or bonds are ok to break, whichever comes first. If ok to break, then each
    # fragment needs to be checked individually again through the loop
    def spf(self, mol, split_id):

        bonds = mol.GetBonds()
        for i in range(len(bonds)):
            if self.okToBreak(bonds[i]):
                mol = Chem.FragmentOnBonds(mol, [i])
                # Dummy atoms are always added last
                n_at = mol.GetNumAtoms()
                print('Split ID', split_id)
                mol.GetAtomWithIdx(n_at-1).SetAtomicNum(split_id)
                mol.GetAtomWithIdx(n_at-2).SetAtomicNum(split_id)
                return Chem.rdmolops.GetMolFrags(mol, asMols=True)

        # If the molecule could not been split, return original molecule
        return [mol]
    #get_fragments(fragment_mols)

    def retreive_plip_interactions(self, pdb_file):
        """
        Retreives the interactions from PLIP.

        Parameters
        ----------
        pdb_file :
            The PDB file of the complex. 

        Returns
        -------
        dict :
            A dictionary of the binding sites and the interactions.
        """
        protlig = PDBComplex()   #instantiate the loader from PLIP
        protlig.load_pdb(pdb_file)   # load the pdb file
        for ligand in protlig.ligands:
            protlig.characterize_complex(ligand)   # find ligands and analyze interactions
        sites = {}
        # loop over binding sites
        for key, site in sorted(protlig.interaction_sets.items()):
            binding_site = BindingSiteReport(site)   # collect data about interactions
            # tuples of *_features and *_info will be converted to pandas DataFrame
            keys = (
                "hydrophobic",
                "hbond",
                "waterbridge",
                "saltbridge",
                "pistacking",
                "pication",
                "halogen",
                "metal"
            )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with 
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')
        # 2. information for each of these features, e.g. for hydrophobic
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')

            interactions = {
                k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
                for k in keys
            }
            sites[key] = interactions
        return sites

    def get_coords_prot(self, RESNR):
        ppdb = PandasPdb()
        ppdb.read_pdb(f"{self.pdb.split('.')[0]}_protein.pdb")
        only_protein=ppdb.df['ATOM']
        resnr_coords=[]
        for i in RESNR:
            resnr_coords.append(list(only_protein[only_protein['atom_number']==int(i)][['x_coord', 'y_coord', 'z_coord']].values[0]))
        return resnr_coords
    
    def interaction_df(self, split):

        all_interactions_df = pd.DataFrame()


        # We create the dictionary for the complex of interest:
        for idx, s in enumerate(split):

            pdb_id=s.split('.')[0]
            raw=pdb_id.split('_')[1]
            idx_frag=int(pdb_id.split('_')[2])
            interactions_by_site = self.retreive_plip_interactions(f"{pdb_id}.pdb")

            # Let’s see how many binding sites are detected:

    #         print(
    #             f"Number of binding sites detected in {pdb_id} : "
    #             f"{len(interactions_by_site)}\n"
    #             f"with {interactions_by_site.keys()}"
    #         )
            # In this case, the first binding site containing ligand 03P will be further investigated.
            index_of_selected_site = 0
            selected_site = list(interactions_by_site.keys())[index_of_selected_site]
            #print(selected_site)


            valid_types = [
                    "hydrophobic",
                    "hbond",
                    "waterbridge",
                    "saltbridge",
                    "pistacking",
                    "pication",
                    "halogen",
                    "metal",
                ]

            for _type in valid_types:
                output_df=self.create_df_from_binding_site(raw, interactions_by_site[selected_site], idx+self.MOL_SPLIT_START, selected_site,
                                                      interactions_by_site,
                                                      interaction_type=_type)
                all_interactions_df=all_interactions_df.append(output_df)
        all_interactions_df = all_interactions_df[all_interactions_df['RESNR'].notna()]
        all_interactions_df.to_csv(f"{self.path}/results_plifs/{raw}_plifs_and_properties.csv", index=False)
        return all_interactions_df


    # We can construct a pandas.DataFrame for a binding site and particular interaction type.

    def create_df_from_binding_site(self, raw, selected_site_interactions, fragment_idx, selected_site, 
                                    interactions_by_site, interaction_type="hbond"):
        """
        Creates a data frame from a binding site and interaction type.

        Parameters
        ----------
        selected_site_interactions : dict
            Precalculated interactions from PLIP for the selected site
        interaction_type : str
            The interaction type of interest (default set to hydrogen bonding).

        Returns
        -------
        pd.DataFrame :
            DataFrame with information retreived from PLIP.
        """
        # check if interaction type is valid:
        valid_types = [
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        ]


        if interaction_type not in valid_types:
            print("!!! Wrong interaction type specified. Hbond is chosen by default !!! \n")
            interaction_type = "hbond"

        def interaction_values(n):
            try:
                interactions=interactions_by_site[selected_site][interaction_type]
                if type(n) is list:
                    return [interactions[1:][x][i] for x in 
                        range(len(interactions[1:])) for i in n]
                else:
                    return [interactions[1:][x][n] for x in 
                        range(len(interactions[1:]))]
            except Exception:
                return None
            
        if interactions_by_site[selected_site][interaction_type][1:]:
            #print(list(map(interaction_values, self.interaction_slices[interaction_type])), self.column_names)
            selected_feats=list(map(interaction_values, self.interaction_slices[interaction_type]))
            #print(selected_feats)
            try: 
                if int(selected_feats[4])>int(selected_feats[3]):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3]  
            except: 
                if int(any(selected_feats[4]))>int(any(selected_feats[3])):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3] 
            df = pd.DataFrame(
                # data is stored AFTER the columns names
                [selected_feats],
                # column names are always the first element - we skipped that in the above - we are gonna use that for naming the df
                columns = self.column_names
            )

            df["INTERACTION_TYPE"]=interaction_type
            
            try:
                checked_coords=self.get_coords_prot(selected_feats[4][0].split(',') if ',' in selected_feats[4][0] \
                                                                   else selected_feats[4])
            except:
                checked_coords=selected_feats[6]
                
            df["AA_COORDS"]=[checked_coords]
                #[self.get_coords_prot(selected_feats[4].split(','))]
            df["FRAGMENT_ATOMS_COORDS"]=[selected_feats[5]]
                            #[self.get_coords_lig(selected_feats[3].split(','))]    
            df['FRAGMENT_ID']=fragment_idx

            # ideally we would like to exclude waters from further processing. Threrfore let us reduce any waterbridge 
            # interaction to the eucladean distance in order to omit water
            
            if interaction_type == "waterbridge":
                df['DIST']=[[np.linalg.norm(x) for x in df['DIST'].to_numpy()]]
                
            # also deal with one distance value and two coords, this is common in saltbridge interactions:
            if len(checked_coords) == len(selected_feats[2])*2:
                df['DIST']=[selected_feats[2] + selected_feats[2]]
                
        else:

            df= pd.DataFrame({'RESNR':[None], 'RESTYPE':[None], 'DIST':[None], 'LIG_IDX':[None],'PROT_IDX':[None],
                        'INTERACTION_TYPE':[interaction_type], "AA_COORDS": [None], "FRAGMENT_ATOMS_COORDS":[None],
                              'FRAGMENT_ID':[str(fragment_idx)]})



        return df
    
    def pdb_2_sdf(self, pdb):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("pdb", "sdf")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

        mol.AddHydrogens()


        obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
        return f"{pdb.split('.')[0]}.sdf"
    
    def sdf_2_pdb(self, sdf):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("sdf", "pdb")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

        mol.AddHydrogens()
        obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
        return f"HETATM_{sdf.split('.')[0]}.pdb"

    def save_bpdb(self, pdb,ppdb, record):  
        ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                    records=[record],
                    gz=False, 
                    append_newline=True)

    def get_HOH_pdb(self, pdb):
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb) 
        ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(self.values)]
        ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=['HETATM'],
                gz=False, 
                append_newline=True)

    def keep_relevant_hetatm(self, pdb):
        raw=str(self.pdb).split('.')[0]
        with open(pdb) as f1, open(f"ATOM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if 'ATOM' in line:
                    f2.write(line)
        with open(f'{raw}_ligand.pdb') as f1, open(f"HETATM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if ('HETATM' in line) and not any(ion in line for ion in self.ions):
                    f2.write(line)
        try: 
            self.get_HOH_pdb(pdb)
        except:
            with open(pdb) as f1, open(f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
                for line in f1:
                    if ('HETATM' in line) and any(ion in line for ion in self.ions):
                        f2.write(line)
        return
    
    
    def fragment_and_plif(self):
        path = os.getcwd()
        if not os.path.exists('results_plifs'):
            os.mkdir(f'{path}/results_plifs')

        raw=str(self.pdb).split('.')[0]
        self.sdf_2_pdb(f'{raw}_ligand.sdf')
        self.keep_relevant_hetatm(f'{raw}_protein.pdb')
        fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
        fragment_mols_alt = Chem.MolFromMol2File(f'{raw}_ligand.mol2', sanitize=True, removeHs=True)
        content = open(f'{raw}_ligand.pdb').read()
        hets=re.findall("^HETATM (.*)", content, re.M)
        if len(hets)<5:
            # Read in the file
            with open(f'{raw}_ligand.pdb', 'r') as file :
                filedata = file.read()

            # Replace the target string
            filedata = filedata.replace('ATOM  ', 'HETATM')

            # Write the file out again
            with open(f'{raw}_ligand.pdb', 'w') as file:
                file.write(filedata)
        
        try: 
            fragment_mols = Chem.RemoveHs(fragment_mols[0])
            output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
            
        except:  
            try:
                fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
                output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
            except:
                try: 
                    output_df = self.interaction_df(self.split_molecule( Chem.MolFromMol2File(fragment_mols_alt)))
                except:
                    try: 
                        fragment_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
                        output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                    except:
                        try:
                            fragment_mols = AllChem.MolFromPDBFile(f'HETATM_{raw}.pdb')
                            output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                        except:
    #                         raise Exception(f"Sorry, the pdb id: {raw} needs chekcing")
                            output_df= pd.DataFrame({'FRAGMENT_ID':[None], 'AA_COORDS':[None], 
                                                     'FRAGMENT_ATOMS_COORDS':[None], 
                                         'INTERACTION_TYPE':[None],'DIST':[None]})
                            print(f"Sorry, the pdb id: {raw} needs chekcing")
        os.chdir(f'{path}')

        return output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST'].agg(list)


def kd_equalizer (value):

    if 'mM' in value.split('=')[1]:
        return float(value.split('m')[0].split('=')[1]) / 1000
    elif 'uM' in value.split('=')[1]:
        return float(value.split('u')[0].split('=')[1]) / 1000000
    elif 'nM' in value.split('=')[1]:
        return float(value.split('n')[0].split('=')[1]) / 1000000000
    elif 'pM' in value.split('=')[1]:
        return float(value.split('p')[0].split('=')[1]) / 1000000000000
    elif 'fM' in value.split('=')[1]:
        return float(value.split('f')[0].split('=')[1]) / 1000000000000000

def dask_plif_cnn_train(row_pdb, row_kd):
    train_grids=None
    train_label=[]
    pdb_id = row_pdb
    print('pdb_id', pdb_id)

    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/{pdb_id}')

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
    
    if not len(df_plifSpecs):
        return

    train_label.extend([row_kd]*10)

    single_pdb_frags = []
    for idx, row in df_plifSpecs.iterrows():

        temp_plifs_prot={}
        temp_plifs_frag={}
        ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
        for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                           row['FRAGMENT_ATOMS_COORDS'],
                                                                                           row['INTERACTION_TYPE'],
                                                                                           row['DIST']):
            # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
            # for two amino acids or ligand atoms! Encoding by atom is crazy fun
            for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                             frag_lig_atm_coord_list):
                temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


        pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
        ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
        single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  

    grid=torch.zeros((full_batch,400,400,400,features_shape))

    for idx, mols in enumerate(single_pdb_frags):
        print(len(single_pdb_frags), idx)
#             threads = multiprocessing.Pool(len(single_pdb_frags))
#             threads.map(func, arg_list)

        coords1, features1 = Feature.get_features(mols[0],mols[2],1)
        coords2, features2 = Feature.get_features(mols[1],mols[3],0)

        # get the center point of protein
        center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
        coords=np.concatenate([coords1,coords2],axis = 0)
        features=np.concatenate([features1,features2],axis = 0)
        assert len(coords) == len(features)
        # zero the coordinates 
        coords = coords-center
        grid=Feature.grid(grid,coords,features,idx, rotation_bool=True)

    if train_grids is None:
        train_grids = grid.to_sparse()
    else:
        train_grids = torch.cat((train_grids,grid.to_sparse()), 0)
    print(train_grids.shape)

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    

    print("Memory utilised (bytes): ", sys.getsizeof(train_grids))
    with open('train_grids.pkl','wb') as f:
        pickle.dump(train_grids, f)
        
    #Save the label data of training and test set.
    with open('train_label.pkl','wb') as f:
        pickle.dump(train_label,f)    
        
    os.chdir(p_directory)
    
    return pdb_id


def dask_plif_cnn_test(row_pdb, row_kd):
    
    test_grids=None
    test_label=[]
    pdb_id = row_pdb
    print('pdb_id', pdb_id)

    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/{pdb_id}')

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
    
    if not len(df_plifSpecs):
        return
    
    test_label.extend([row_kd])

    single_pdb_frags = []
    print(len(single_pdb_frags))
    for idx, row in df_plifSpecs.iterrows():

        temp_plifs_prot={}
        temp_plifs_frag={}
        ## do something with fragment_idx . i.e. open the pdb and do your shit with encoding\
        for aa_atm_coord_list, frag_lig_atm_coord_list, interaction, dist_list in zip (row['AA_COORDS'], 
                                                                                           row['FRAGMENT_ATOMS_COORDS'],
                                                                                           row['INTERACTION_TYPE'],
                                                                                           row['DIST']):
            # because sometimes salt bridges makes two concurrent connections so it is possible that we have one distance
            # for two amino acids or ligand atoms! Encoding by atom is crazy fun
            for dist, aa_atm_coord, frag_lig_atm_coord in zip (dist_list, aa_atm_coord_list, 
                                                                             frag_lig_atm_coord_list):
                temp_plifs_prot[tuple(aa_atm_coord)]=[interaction,dist]
                temp_plifs_frag[tuple(frag_lig_atm_coord)]=[interaction,dist]


        pdb = next(pybel.readfile('pdb',os.path.join(path,'ATOM_' + pdb_id + '.pdb')))
        ligand = next(pybel.readfile('pdb',os.path.join(path, pdb_id + f'_{str(idx)}'+'.pdb')))
        single_pdb_frags.append((pdb,ligand,temp_plifs_prot,temp_plifs_frag))  

    grid=torch.zeros((1,400,400,400,features_shape))

    for idx, mols in enumerate(single_pdb_frags):
        coords1, features1 = Feature.get_features(mols[0],mols[2],1)
        coords2, features2 = Feature.get_features(mols[1],mols[3],0)

        # get the center point of protein
        center=(np.max(coords2,axis=0)+np.min(coords2,axis=0))/2
        coords=np.concatenate([coords1,coords2],axis = 0)
        features=np.concatenate([features1,features2],axis = 0)
        assert len(coords) == len(features)
        # zero the coordinates 
        coords = coords-center
        grid=Feature.grid(grid,coords,features,idx, rotation_bool=False)

    if test_grids is None:
        test_grids = grid.to_sparse()
    else:
        test_grids = torch.cat((test_grids,grid.to_sparse()), 0)
    print(test_grids.shape)

    raw=pdb_id
    path = os.getcwd()
    fileList = []
    fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
    fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
    fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    


    with open('test_grids.pkl','wb') as f:
        pickle.dump(test_grids, f)
        
    #Save the label data of training and test set.
    with open('test_label.pkl','wb') as f:
        pickle.dump(test_label,f)    
        
    os.chdir(p_directory)
    
    return pdb_id

if __name__ == "__main__":
    
    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/')

    Feature = Feature_extractor()

    p_directory = os.getcwd()

#     general=pd.read_csv('INDEX_general_PL_data.2020', sep=',')
#     refined=pd.read_csv('INDEX_refined_data.2020', sep=',')

#     general=general[general["Kd/Ki"].str.contains('IC|EC|>|<')==False]
#     refined=refined[refined["Kd/Ki"].str.contains('IC|EC|>|<')==False]

#     general["Kd/Ki"] = general["Kd/Ki"].str.replace('~','=')
#     refined["Kd/Ki"] = refined["Kd/Ki"].str.replace('~','=')


#     general['Kd/Ki']=general['Kd/Ki'].apply(lambda x: kd_equalizer(x))
#     refined['Kd/Ki']=refined['Kd/Ki'].apply(lambda x: kd_equalizer(x))

#     merged_PDBBind=general.append(refined) \
#                                 .sample(frac=1) \
#                                 .sample(frac=1) \
#                                 .reset_index(drop=True) \
#                                 .drop_duplicates(subset='PDB_code', keep="first") 

    merged_PDBBind=pd.read_csv('merged_PDBBind.csv')
#     merged_PDBBind.rename(columns={'Kd/Ki': 'Kd_Ki'}, inplace=True)
#     merged_PDBBind[merged_PDBBind['PDB_code'].str.contains("3bho")==False]

    train_df, test_df = train_test_split(merged_PDBBind, test_size=0.1)

#     merged_PDBBind.to_csv('merged_PDBBind.csv', index=False)
    train_df.to_csv('train_df.csv', index=False)
    test_df.to_csv('test_df.csv', index=False)


    train_df.parallel_apply(lambda x: dask_plif_cnn_train(x.PDB_code, x.Kd_Ki), axis=1)
    test_df.parallel_apply(lambda x: dask_plif_cnn_test(x.PDB_code, x.Kd_Ki), axis=1)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
pdb_idpdb_idpdb_idpdb_id pdb_id 5nea 
pdb_id pdb_id 3b28pdb_idpdb_idpdb_id1txr1a4r 
 6sge

 
6qfu 
3lpgpdb_id
 2n0w pdb_idpdb_idpdb_idpdb_id5orvpdb_idpdb_idpdb_id

pdb_id16pk1azm   pdb_id    
pdb_id pdb_id
5q0h1uh1 4aze4j092h4k 

3o7u1g6s2ay3

5tco
 pdb_id

2xaf

 6b5a

5m4qpdb_id
 1a7t


*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 6sge_ligand)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 6qfu_ligand)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 5q0h_ligand)



Split IDSplit ID 70


[20:58:20] 6sge_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.


Split ID

...

Split ID 70
Split ID 71
Split ID 72
Split ID 73
4 0
4 3
4 1
torch.Size([10, 400, 400, 400, 74])
Memory utilised (bytes):  72
4 2
4 3
4 0
4 1
4 2
pdb_id 6hm1
Split ID 70
Split ID 71
Split ID 72
torch.Size([10, 400, 400, 400, 74])Split ID 
Memory utilised (bytes): 73
 72Split ID
 74
Split ID 75
Split ID 76
Split ID 77
Split ID 78
Split ID 79
pdb_id 2flb
4 3


[21:39:15] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 9 10 11


Split ID 70
Split ID 71
Split ID 72
torch.Size([10, 400, 400, 400, 74])
Memory utilised (bytes):  72
pdb_id 5wuk
Split ID 70
Split ID 71
Split ID 723
Split ID  073

5 0
3 1
5 1
5 2
3 2
torch.Size([10, 400, 400, 400, 74])
Memory utilised (bytes):  72
pdb_id 6a05


[21:39:40] 6a05_ligand: warning - O.co2 with non C.2 or S.o2 neighbor.
[21:39:40] Can't kekulize mol.  Unkekulized atoms: 2 6 7 8 9


Split ID 70
Split ID 71
Split ID 72
Split ID5  733

5 4
3 0
3 1
3 2


In [32]:
from openbabel import pybel

In [29]:
output_df= pd.DataFrame({'FRAGMENT_ID':[None], 'AA_COORDS':[None], 
                         'FRAGMENT_ATOMS_COORDS':[None], 
             'INTERACTION_TYPE':[None],'DIST':[None]})

output_df=output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST'].agg(list)

In [30]:
len(output_df)

0

In [2]:
os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/')

In [3]:
merged_PDBBind.to_csv('merged_PDBBind.csv', index=False)

In [12]:
ddf1.iloc[:, 1:4]

,resolution,release_year,-logKd/Ki
npartitions=1000,,,
1,object,int64,float64
12,...,...,...
...,...,...,...
17357,...,...,...
17380,...,...,...


In [16]:
def myadd(row):
    return print('hi',row)

ddf1.apply(lambda x: myadd(x.PDB_ID), axis=1)

,PDB_code,resolution,release_year,-logKd/Ki,Kd/Ki,reference,ligand_name
npartitions=1000,,,,,,,
0,object,object,int64,float64,float64,object,object
13,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
17360,...,...,...,...,...,...,...
17379,...,...,...,...,...,...,...


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [27]:
ddf1 = train_df
ddf2 = test_df

In [29]:
ddf1.divisions

(0,
 12,
 26,
 39,
 52,
 64,
 76,
 89,
 103,
 114,
 126,
 137,
 149,
 162,
 173,
 185,
 196,
 207,
 219,
 230,
 242,
 255,
 267,
 279,
 292,
 303,
 314,
 326,
 338,
 350,
 361,
 374,
 385,
 397,
 409,
 420,
 434,
 447,
 461,
 474,
 486,
 498,
 512,
 525,
 537,
 549,
 561,
 572,
 584,
 597,
 609,
 621,
 632,
 643,
 655,
 667,
 681,
 694,
 705,
 719,
 732,
 743,
 754,
 768,
 781,
 797,
 811,
 823,
 838,
 850,
 861,
 872,
 884,
 896,
 908,
 920,
 933,
 946,
 957,
 968,
 982,
 995,
 1006,
 1018,
 1032,
 1043,
 1057,
 1071,
 1083,
 1095,
 1108,
 1121,
 1132,
 1144,
 1156,
 1169,
 1184,
 1197,
 1210,
 1227,
 1240,
 1254,
 1268,
 1280,
 1293,
 1308,
 1321,
 1332,
 1345,
 1359,
 1371,
 1388,
 1401,
 1415,
 1428,
 1440,
 1454,
 1469,
 1480,
 1491,
 1505,
 1522,
 1533,
 1546,
 1558,
 1571,
 1584,
 1595,
 1609,
 1624,
 1639,
 1653,
 1669,
 1681,
 1694,
 1707,
 1720,
 1732,
 1748,
 1762,
 1773,
 1786,
 1797,
 1808,
 1822,
 1836,
 1851,
 1863,
 1876,
 1888,
 1902,
 1916,
 1929,
 1943,
 1956,
 1967,

In [3]:
train_df

,PDB_code,resolution,release_year,-logKd/Ki,Kd/Ki,reference,ligand_name
17181,4ccd,1.97,2013,4.49,3.200000e-05,4ccc.pdf,(2DG)
11372,5gv2,2.06,2017,5.65,2.230000e-06,5gv2.pdf,(ARG)
10961,5vtb,2.40,2017,6.96,1.100000e-07,5vtb.pdf,(15-mer)
2912,3ui7,2.28,2011,9.00,1.000000e-09,3ui7.pdf,(C1L)
12060,5kbf,2.00,2016,8.92,1.200000e-09,5k8s.pdf,(CMP)
...,...,...,...,...,...,...,...
12038,1t3t,1.90,2004,4.96,1.100000e-05,1t3t.pdf,(ADP)
5256,3u7m,2.15,2012,7.45,3.510000e-08,3u7m.pdf,(FHF)
1158,3h91,1.50,2009,3.73,1.850000e-04,2l11.pdf,(15-mer)
13065,2i3v,2.40,2006,6.55,2.820000e-07,2i3v.pdf,(GLU)


In [25]:
from dask.dataframe import from_pandas
ddf = from_pandas(test_df, npartitions=5, sort=True)  # Dask DataFrame has 5 partitions

In [26]:
ddf

,PDB_code,resolution,release_year,-logKd/Ki,Kd/Ki,reference,ligand_name
npartitions=5,,,,,,,
1,object,object,int64,float64,float64,object,object
2507,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
12427,...,...,...,...,...,...,...
17382,...,...,...,...,...,...,...


In [1]:
import numpy as np
import mpi4py
import time

In [ ]:
from mpi4py import MPI

In [6]:
from os import environ 
from dask_mpi import initialize
from distributed import Client
from datetime import datetime
from time import sleep
from random import random

# Work out from the environment how many threads to allocate
num_threads = int(environ.get(
    'SLURM_CPUS_PER_TASK',
    environ.get('OMP_NUM_THREADS', 1)
))

# Create the Dask workers
# initialize(interface='cn30', nthreads=num_threads)

# Create the Dask object that will manage the communications
client = Client()

start = datetime.now()
# run_test(client=client)
end = datetime.now()

print(f"Time taken: {end - start}")

Time taken: 0:00:00.000060


In [7]:
print(client)

<Client: 'tcp://127.0.0.1:40583' processes=4 threads=16, memory=32.00 GiB>


In [1]:
from dask_mpi import initialize

In [3]:
pip install dask_mpi

Note: you may need to restart the kernel to use updated packages.


In [8]:
import multiprocessing



In [ ]:
train_grids[train_grids is 'nan'] = 0

In [7]:
train_grids.to_dense()

RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1290240000000 bytes. Error code 12 (Cannot allocate memory)

In [5]:
print("Memory utilised (bytes): ", sys.getsizeof(train_grids))

Memory utilised (bytes):  72


In [3]:
import torch
grid=torch.zeros((10,400,400,400,4))

In [4]:
grid2=torch.zeros((10,400,400,400,4))

In [5]:
grid2.shape[1:]

torch.Size([400, 400, 400, 4])

In [15]:
dim=grid.shape[1:]
print(dim)
grid3= torch.zeros((grid.shape[0] + grid2.shape[0], (400, 400, 400, 4)))

torch.Size([400, 400, 400, 4])


TypeError: zeros(): argument 'size' must be tuple of ints, but found element of type tuple at pos 2

In [6]:
grid3=torch.cat((grid.to_sparse(), grid2.to_sparse()), 0)

In [7]:
grid3.shape

torch.Size([20, 400, 400, 400, 4])

In [8]:
grid3

tensor(indices=tensor([[ 2,  2, 12, 12],
                       [ 0,  0,  0,  0],
                       [16, 16, 16, 16],
                       [ 3,  3,  3,  3],
                       [ 0,  2,  0,  2]]),
       values=tensor([1., 1., 1., 1.]),
       size=(20, 400, 400, 400, 4), nnz=4, layout=torch.sparse_coo)

In [16]:
import sys
print("Memory utilised (bytes): ", sys.getsizeof(grid3))

Memory utilised (bytes):  72


In [5]:
grid[2,0,16,3] += torch.tensor([1.,0.,1,0.])
grid2[2,0,16,3] += torch.tensor([1.,0.,1,0.])

In [ ]:
grid[2,0,16,3]

In [ ]:
grid.shape

In [ ]:
train_grids.shape

In [ ]:
import numpy as np
from scipy import sparse

In [ ]:
grid=np.zeros((100000,400,400,400,72),dtype=np.float16)

In [ ]:
b=sparse.csr_matrix(train_grids.ravel())

In [ ]:
print("Memory utilised (bytes): ", sys.getsizeof(b))

In [ ]:
b_dense = b.todense()
b_dense.shape

In [ ]:
import torch
b_d_tensor = torch.Tensor(b_dense)

In [ ]:
b_d_tensor.shape

In [ ]:
bd_right_shape_tensor = b_d_tensor.reshape((10, 400, 400, 400, 72))

In [ ]:
print("Memory utilised (bytes): ", sys.getsizeof(bd_right_shape_tensor))

In [ ]:
bd_right_shape_tensor.shape

In [ ]:
train_grids.shape

In [15]:
import warnings
import requests
import os
import re
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
import oddt
from oddt import fingerprints
warnings.filterwarnings("ignore")

class PLIF:
    def __init__(self, PDB: str, MOL_SPLIT_START: int = 70, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super(PLIF,self).__init__()
        
        self.MOL_SPLIT_START=MOL_SPLIT_START
        self.pdb=PDB
        self.records=['ATOM']
        self.values=['HOH','CL','MG','ZN','MN','CA']
        self.ions=['CL','MG','ZN','MN','CA']
        self.interaction_slices={"hydrophobic":[0,1,6,7,8,9,10],
            "hbond":[0,1,7,11,13,15,16],
            "waterbridge":[0,1,[6,7],11,13,16,17],
            "saltbridge":[0,1,7,10,3,11,12],
            "pistacking":[0,1,7,11,6,12,13],
            "pication":[0,1,7,11,3,12,13],
            "halogen":[0,1,7,10,12,14,15],
            "metal":[0,1,11,8,6,17,16]} 

        self.column_names = ['RESNR', 'RESTYPE', 'DIST', 'LIG_IDX','PROT_IDX','FRAGMENT_ATOMS_COORDS', 'AA_COORDS']
        self.path = os.getcwd()


    def okToBreak(self, bond):
        """
        Here we apply a bunch of rules to judge if the bond is OK to break.

        Parameters
        ----------
        bond :
            RDkit MOL object

        Returns
        -------
        Boolean :
            OK or not to break.
        """
        # See if the bond is in Ring (don't break that)
        if bond.IsInRing():
            return False
        # We OK only single bonds to break
        if bond.GetBondType() != Chem.rdchem.BondType.SINGLE:
            return False

        # Get the beginning atom of the bond
        begin_atom = bond.GetBeginAtom()
        # Get the ending atom of the bond
        end_atom = bond.GetEndAtom()
        # What kind of neighbors does these end and begenning atoms have? We need a family of no less than 5!
        neighbor_end=list(end_atom.GetNeighbors())
        neighbor_begin=list(begin_atom.GetNeighbors())
        if (len(neighbor_end) + len(neighbor_begin)) <5:
            return False
        #for atm in neighbor_end:
            #print(atm.GetAtomicNum())
        #print(begin_atom.GetAtomicNum(), end_atom.GetAtomicNum(), MOL_SPLIT_START)
        
        # Now check if end or begenning atoms are in ring (we dont wanna bother those)
        if not(begin_atom.IsInRing() or end_atom.IsInRing()):
            return False
        elif begin_atom.GetAtomicNum() >= self.MOL_SPLIT_START or \
                end_atom.GetAtomicNum() >= self.MOL_SPLIT_START:
            return False
        elif end_atom.GetAtomicNum() == 1:
            return False
        else:
            return True

    def undo_id_label (self, frag, split_id):
        # I am trying to restore Hydrogens where the break happened
        for i, atom in enumerate(frag.GetAtoms()):
            if atom.GetAtomicNum() >= split_id:
                atom.SetAtomicNum(1)

        return frag

    # Divide a molecule into fragments
    def split_molecule(self, mol, pdb):

        split_id = self.MOL_SPLIT_START

        res = []
        res_no_id=[]

        to_check = [mol]
        while len(to_check) > 0:
            ms = self.spf(to_check.pop(), split_id)
            if len(ms) == 1:
                res += ms
            else:
                to_check += ms
                split_id += 1
        for frag in res:
            res_no_id.append(self.undo_id_label(frag, self.MOL_SPLIT_START))

        res_pdb_frags=[]

        for idx, frag in enumerate(res_no_id):
            w = Chem.PDBWriter(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
            w.write(frag)
            w.close()
            
            unwanted_entries= ['CONECT', 'END']            
            with open(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as oldfile, open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as newfile:
                for line in oldfile:
                    if not any(unwanted_entry in line for unwanted_entry in unwanted_entries):
                        newfile.write(line)

                    
            data = data2 = ""

            # Reading data from file1
            with open(f"ATOM_{pdb}.pdb") as fp:
                data = fp.read()

            # Reading data from file2

            with open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as fp:
                data2 = fp.read()
            
            # Merging 2 files
            # To add the data of file2
            # from next line
            #data += "\n"
            data += data2
            
            with open(f"HOH_{pdb}.pdb") as fp:
                data3 = fp.read()
            data += data3

            with open (f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as fp:
                fp.write(data)
            res_pdb_frags.append(f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        return res_pdb_frags #create_chain(res)


    # Function for doing all the nitty gritty splitting work.
    # loops over bonds until bonds get exhausted or bonds are ok to break, whichever comes first. If ok to break, then each
    # fragment needs to be checked individually again through the loop
    def spf(self, mol, split_id):

        bonds = mol.GetBonds()
        for i in range(len(bonds)):
            if self.okToBreak(bonds[i]):
                mol = Chem.FragmentOnBonds(mol, [i])
                # Dummy atoms are always added last
                n_at = mol.GetNumAtoms()
                print('Split ID', split_id)
                mol.GetAtomWithIdx(n_at-1).SetAtomicNum(split_id)
                mol.GetAtomWithIdx(n_at-2).SetAtomicNum(split_id)
                return Chem.rdmolops.GetMolFrags(mol, asMols=True)

        # If the molecule could not been split, return original molecule
        return [mol]
    #get_fragments(fragment_mols)

    def retreive_plip_interactions(self, pdb_file):
        """
        Retreives the interactions from PLIP.

        Parameters
        ----------
        pdb_file :
            The PDB file of the complex. 

        Returns
        -------
        dict :
            A dictionary of the binding sites and the interactions.
        """
        protlig = PDBComplex()   #instantiate the loader from PLIP
        protlig.load_pdb(pdb_file)   # load the pdb file
        for ligand in protlig.ligands:
            protlig.characterize_complex(ligand)   # find ligands and analyze interactions
        sites = {}
        # loop over binding sites
        for key, site in sorted(protlig.interaction_sets.items()):
            binding_site = BindingSiteReport(site)   # collect data about interactions
            # tuples of *_features and *_info will be converted to pandas DataFrame
            keys = (
                "hydrophobic",
                "hbond",
                "waterbridge",
                "saltbridge",
                "pistacking",
                "pication",
                "halogen",
                "metal"
            )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with 
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')
        # 2. information for each of these features, e.g. for hydrophobic
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')

            interactions = {
                k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
                for k in keys
            }
            sites[key] = interactions
        return sites

    def get_coords_prot(self, RESNR):
        ppdb = PandasPdb()
        ppdb.read_pdb(f"{self.pdb.split('.')[0]}_protein.pdb")
        only_protein=ppdb.df['ATOM']
        resnr_coords=[]
        for i in RESNR:
            resnr_coords.append(list(only_protein[only_protein['atom_number']==int(i)][['x_coord', 'y_coord', 'z_coord']].values[0]))
        return resnr_coords
    
    def interaction_df(self, split):

        all_interactions_df = pd.DataFrame()


        # We create the dictionary for the complex of interest:
        for idx, s in enumerate(split):

            pdb_id=s.split('.')[0]
            raw=pdb_id.split('_')[1]
            idx_frag=int(pdb_id.split('_')[2])
            interactions_by_site = self.retreive_plip_interactions(f"{pdb_id}.pdb")

            # Let’s see how many binding sites are detected:

    #         print(
    #             f"Number of binding sites detected in {pdb_id} : "
    #             f"{len(interactions_by_site)}\n"
    #             f"with {interactions_by_site.keys()}"
    #         )
            # In this case, the first binding site containing ligand 03P will be further investigated.
            index_of_selected_site = 0
            selected_site = list(interactions_by_site.keys())[index_of_selected_site]
            #print(selected_site)


            valid_types = [
                    "hydrophobic",
                    "hbond",
                    "waterbridge",
                    "saltbridge",
                    "pistacking",
                    "pication",
                    "halogen",
                    "metal",
                ]

            for _type in valid_types:
                output_df=self.create_df_from_binding_site(raw, interactions_by_site[selected_site], idx+self.MOL_SPLIT_START, selected_site,
                                                      interactions_by_site,
                                                      interaction_type=_type)
                all_interactions_df=all_interactions_df.append(output_df)
        all_interactions_df = all_interactions_df[all_interactions_df['RESNR'].notna()]
        #all_interactions_df.to_csv(f"{self.path}/results_plifs/{raw}_plifs_and_properties.csv", index=False)
        return all_interactions_df


    # We can construct a pandas.DataFrame for a binding site and particular interaction type.

    def create_df_from_binding_site(self, raw, selected_site_interactions, fragment_idx, selected_site, 
                                    interactions_by_site, interaction_type="hbond"):
        """
        Creates a data frame from a binding site and interaction type.

        Parameters
        ----------
        selected_site_interactions : dict
            Precalculated interactions from PLIP for the selected site
        interaction_type : str
            The interaction type of interest (default set to hydrogen bonding).

        Returns
        -------
        pd.DataFrame :
            DataFrame with information retreived from PLIP.
        """
        # check if interaction type is valid:
        valid_types = [
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        ]


        if interaction_type not in valid_types:
            print("!!! Wrong interaction type specified. Hbond is chosen by default !!! \n")
            interaction_type = "hbond"

        def interaction_values(n):
            try:
                interactions=interactions_by_site[selected_site][interaction_type]
                if type(n) is list:
                    return [interactions[1:][x][i] for x in 
                        range(len(interactions[1:])) for i in n]
                else:
                    return [interactions[1:][x][n] for x in 
                        range(len(interactions[1:]))]
            except Exception:
                return None
            
        if interactions_by_site[selected_site][interaction_type][1:]:
            #print(list(map(interaction_values, self.interaction_slices[interaction_type])), self.column_names)
            selected_feats=list(map(interaction_values, self.interaction_slices[interaction_type]))
            #print(selected_feats)
            try: 
                if int(selected_feats[4])>int(selected_feats[3]):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3]  
            except: 
                if int(any(selected_feats[4]))>int(any(selected_feats[3])):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3] 
            df = pd.DataFrame(
                # data is stored AFTER the columns names
                [selected_feats],
                # column names are always the first element - we skipped that in the above - we are gonna use that for naming the df
                columns = self.column_names
            )

            df["INTERACTION_TYPE"]=interaction_type
            
            try:
                checked_coords=self.get_coords_prot(selected_feats[4][0].split(',') if ',' in selected_feats[4][0] \
                                                                   else selected_feats[4])
            except:
                checked_coords=selected_feats[6]
                
            df["AA_COORDS"]=[checked_coords]
                #[self.get_coords_prot(selected_feats[4].split(','))]
            df["FRAGMENT_ATOMS_COORDS"]=[selected_feats[5]]
                            #[self.get_coords_lig(selected_feats[3].split(','))]    
            df['FRAGMENT_ID']=fragment_idx

            # ideally we would like to exclude waters from further processing. Threrfore let us reduce any waterbridge 
            # interaction to the eucladean distance in order to omit water
            
            if interaction_type == "waterbridge":
                df['DIST']=[[np.linalg.norm(x) for x in df['DIST'].to_numpy()]]
                
            # also deal with one distance value and two coords, this is common in saltbridge interactions:
            if len(checked_coords) == len(selected_feats[2])*2:
                df['DIST']=[selected_feats[2] + selected_feats[2]]
                
        else:

            df= pd.DataFrame({'RESNR':[None], 'RESTYPE':[None], 'DIST':[None], 'LIG_IDX':[None],'PROT_IDX':[None],
                        'INTERACTION_TYPE':[interaction_type], "AA_COORDS": [None], "FRAGMENT_ATOMS_COORDS":[None],
                              'FRAGMENT_ID':[str(fragment_idx)]})



        return df
    
    def pdb_2_sdf(self, pdb):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("pdb", "sdf")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

        mol.AddHydrogens()


        obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
        return f"{pdb.split('.')[0]}.sdf"
    
    def sdf_2_pdb(self, sdf):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("sdf", "pdb")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

        mol.AddHydrogens()
        obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
        return f"HETATM_{sdf.split('.')[0]}.pdb"

    def save_bpdb(self, pdb,ppdb, record):  
        ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                    records=[record],
                    gz=False, 
                    append_newline=True)

    def get_HOH_pdb(self, pdb):
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb) 
        ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(self.values)]
        ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=['HETATM'],
                gz=False, 
                append_newline=True)

    def keep_relevant_hetatm(self, pdb):
        raw=str(self.pdb).split('.')[0]
        with open(pdb) as f1, open(f"ATOM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if 'ATOM' in line:
                    f2.write(line)
        with open(f'{raw}_ligand.pdb') as f1, open(f"HETATM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if ('HETATM' in line) and not any(ion in line for ion in self.ions):
                    f2.write(line)
        try: 
            self.get_HOH_pdb(pdb)
        except:
            with open(pdb) as f1, open(f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
                for line in f1:
                    if ('HETATM' in line) and any(ion in line for ion in self.ions):
                        f2.write(line)
        return
    
    
    def fragment_and_plif(self):
        path = os.getcwd()
        if not os.path.exists('results_plifs'):
            os.mkdir(f'{path}/results_plifs')

        raw=str(self.pdb).split('.')[0]
        self.sdf_2_pdb(f'{raw}_ligand.sdf')
        self.keep_relevant_hetatm(f'{raw}_protein.pdb')
        fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
        fragment_mols_alt = Chem.MolFromMol2File(f'{raw}_ligand.mol2', sanitize=True, removeHs=True)
        content = open(f'{raw}_ligand.pdb').read()
        hets=re.findall("^HETATM (.*)", content, re.M)
        if len(hets)<5:
            # Read in the file
            with open(f'{raw}_ligand.pdb', 'r') as file :
                filedata = file.read()

            # Replace the target string
            filedata = filedata.replace('ATOM  ', 'HETATM')

            # Write the file out again
            with open(f'{raw}_ligand.pdb', 'w') as file:
                file.write(filedata)
        
        try: 
            fragment_mols = Chem.RemoveHs(fragment_mols[0])
            output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
            
        except:  
            try: 
                output_df = self.interaction_df(self.split_molecule(fragment_mols_alt))
                
            except:
                try: 
                    fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
                    output_df = self.interaction_df(self.split_molecule(fragment_mols[0],raw))
                except:
                    try:
                        fragment_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
                        output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                    except:
                        try:
                            fragment_mols = AllChem.MolFromPDBFile(f'HETATM_{raw}.pdb')
                            output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
                        except:
#                             raise Exception(f"Sorry, the pdb id: {raw} needs chekcing")
                            output_df= pd.DataFrame({'FRAGMENT_ID':[None], 'AA_COORDS':[None], 
                                                     'FRAGMENT_ATOMS_COORDS':[None], 
                                         'INTERACTION_TYPE':[None],'DIST':[None]})
        os.chdir(f'{path}')

        return output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST'].agg(list)

if __name__ == "__main__":
    
    p_directory = os.getcwd()
    directory='/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/*'
#     file_list=[pdb_id.split('/')[-1] for pdb_id in glob.glob(directory)]
#     for pdb_id in file_list:
    pdb_id = '5va9'
    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/{pdb_id}')
#     os.chdir(f'/PDBbind_v2020_refined/refined_set/{pdb_id}')

    df_plifSpecs = PLIF(PDB = f'{pdb_id}.pdb').fragment_and_plif()
    
    if not len(df_plifSpecs) or df_plifSpecs['INTERACTION_TYPE'].apply(', '.join).unique()[0]=='metal':
        print(f"Sorry, the pdb id: {pdb_id} needs chekcing")

[21:44:10] Can't kekulize mol.  Unkekulized atoms: 24 25 26 27 28
[21:44:10] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20
[21:44:10] Explicit valence for atom # 101 Rn greater than permitted


Split ID 70
Split ID 70
Split ID 71
Split ID 72
Split ID 73
Split ID 74
Split ID 75
Split ID 76
Split ID 77
Split ID 78
Split ID 79
Split ID 80
Split ID 81
Split ID 82
Split ID 83
Split ID 84
Split ID 85
Split ID 86
Split ID 70
Sorry, the pdb id: 5va9 needs chekcing


In [12]:
df_plifSpecs['INTERACTION_TYPE'].apply(', '.join).unique()[0]

'metal'

In [6]:
df_plifSpecs

,AA_COORDS,FRAGMENT_ATOMS_COORDS,INTERACTION_TYPE,DIST
FRAGMENT_ID,,,,
70,"[[(61.653, 29.816, 11.629), (61.653, 29.816, 1...","[[(61.774, 30.976, 13.683), (61.881, 27.528, 1...",[metal],"[[2.36, 2.73, 2.33, 2.67, 2.28]]"
71,"[[(61.653, 29.816, 11.629), (61.653, 29.816, 1...","[[(61.774, 30.976, 13.683), (61.881, 27.528, 1...",[metal],"[[2.36, 2.73, 2.33, 2.67, 2.28]]"


In [2]:
import glob
import warnings
import requests
import os
import re
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
import oddt
from oddt import fingerprints
warnings.filterwarnings("ignore")

records=['ATOM']
values=['HOH','CL','MG','ZN','MN','CA']
ions=['CL','MG','ZN','MN','CA']

def pdb_2_sdf(pdb):
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("pdb", "sdf")
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

    mol.AddHydrogens()


    obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
    return f"{pdb.split('.')[0]}.sdf"

def sdf_2_pdb(sdf):
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("sdf", "pdb")
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

    mol.AddHydrogens()
    obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
    return f"HETATM_{sdf.split('.')[0]}.pdb"

def save_bpdb(pdb,ppdb, record):  
    ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=[record],
                gz=False, 
                append_newline=True)

def get_HOH_pdb(pdb):
    ppdb = PandasPdb() 
    ppdb.read_pdb(pdb) 
    ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(values)]
    ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
            records=['HETATM'],
            gz=False, 
            append_newline=True)

def keep_relevant_hetatm(raw, pdb):
    with open(pdb) as f1, open(f"ATOM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
        for line in f1:
            if 'ATOM' in line:
                f2.write(line)
    with open(f'{raw}_ligand.pdb') as f1, open(f"HETATM_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
        for line in f1:
            if ('HETATM' in line) and not any(ion in line for ion in ions):
                f2.write(line)
    try: 
        get_HOH_pdb(pdb)
    except:
        with open(pdb) as f1, open(f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb", 'w') as f2:
            for line in f1:
                if ('HETATM' in line) and any(ion in line for ion in ions):
                    f2.write(line)
    return
os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set_cp/')
p_directory = os.getcwd()
counter=0
directory='/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set_cp/*'
file_list=[pdb_id.split('/')[-1] for pdb_id in glob.glob(directory)]
for pdb_id in file_list:
    print(pdb_id)
    os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set_cp/{pdb_id}')

    raw=pdb_id
    sdf_2_pdb(f'{raw}_ligand.sdf')
    keep_relevant_hetatm(raw, f'{pdb_id}_protein.pdb')
    fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
    fragment_mols_alt = Chem.MolFromMol2File(f'{raw}_ligand.mol2', sanitize=True, removeHs=True)
    content = open(f'{raw}_ligand.pdb').read()
    hets=re.findall("^HETATM (.*)", content, re.M)
    if len(hets)<5:
        # Read in the file
        with open(f'{raw}_ligand.pdb', 'r') as file :
            filedata = file.read()

        # Replace the target string
        filedata = filedata.replace('ATOM  ', 'HETATM')

        # Write the file out again
        with open(f'{raw}_ligand.pdb', 'w') as file:
            file.write(filedata)
#     os.chdir(f'/PDBbind_v2020_refined/refined_set/{pdb_id}')
    try: 
        assert Chem.RemoveHs(fragment_mols[0]) is not None
        print(fragment_mols.GetNumAtoms())
    except:  
        try: 
            assert fragment_mols_alt is not None
            fragment_mols_alt.GetBonds()
            print(fragment_mols_alt.GetNumAtoms())
        except:
            try: 
                assert fragment_mols[0] is not None
                fragment_mols[0].GetBonds()
                print(fragment_mols[0].GetNumAtoms())
            except:
                try:
                    fragment_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
                    assert fragment_mols is not None
                    fragment_mols.GetBonds()
                    print(fragment_mols.GetNumAtoms())
                except:
                    try:
                        fragment_mols = AllChem.MolFromPDBFile(f'HETATM_{raw}.pdb')
                        assert fragment_mols is not None
                        fragment_mols.GetBonds()
                        print(fragment_mols.GetNumAtoms())
                    except:
                         print(f"Sorry, the pdb id: {raw} needs chekcing")
    counter+=1
    #print(counter)

    os.chdir(f'{p_directory}')
    

2lbv
22
1lt6
21
4eoh
13
3zc5


[15:20:37] Can't kekulize mol.  Unkekulized atoms: 6 7 10 11 12


27
6hqy
22
1utl
10
4lwi
30
3t4p


[15:20:38] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21


37
6oyz
40
5y13
22
4i11
24
5i2e
38
6n0q
31
2vvo


...

[15:22:08] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16


41
5aom
17
6dn6
38
6qha
22
2vo5
20
1fo3
16
3ia6
31
5tz3
29
4irx
12
2w3i
40
2bvr
27
3ral
27
2baj
27
1kzn


[15:22:11] Can't kekulize mol.  Unkekulized atoms: 39 40 41 42 43


49
4a6v
19
3u7n
27
2wks
24
1h46


KeyboardInterrupt: 

In [36]:
file_list

['2lbv',
 '1lt6',
 '4eoh',
 '3zc5',
 '6hqy',
 '1utl',
 '4lwi',
 '3t4p',
 '6oyz',
 '5y13',
 '4i11',
 '5i2e',
 '6n0q',
 '2vvo',
 '4tk1',
 '1p1n',
 '5l2w',
 '2yay',
 '2y7i',
 '1i7g',
 '1nj5',
 '5wei',
 '3l7d',
 '4v04',
 '4qmz',
 '6f3e',
 '3vby',
 '4b7q',
 '6qme',
 '1ft7',
 '2xln',
 '2oz2',
 '5ih9',
 '5u0w',
 '2ga2',
 '6eox',
 '5ypp',
 '5z95',
 '4a4l',
 '4efk',
 '4us3',
 '4alv',
 '5he4',
 '5jrq',
 '3k97',
 '3cvk',
 '3qck',
 '1uyi',
 '3oy0',
 '2w67',
 '5ivz',
 '4c1y',
 '3rr4',
 '1mnc',
 '4gqr',
 '5ur1',
 '5qc4',
 '1apv',
 '6cwh',
 '6qyp',
 '1o3f',
 '1szm',
 '2xxy',
 '1hxw',
 '4dv8',
 '5kly',
 '3hxb',
 '4mvn',
 '3u3u',
 '4pax',
 '5j8x',
 '1c5y',
 '5a14',
 '5cas',
 '5c7c',
 '4a9i',
 '5ukj',
 '5ab0',
 '4zuq',
 '3juo',
 '6hke',
 '6g9x',
 '2p33',
 '5g60',
 '4g0k',
 '6kdi',
 '2c4w',
 '1ru2',
 '4npv',
 '4rxe',
 '3ck8',
 '5abw',
 '4uco',
 '6f6s',
 '4lq3',
 '6cnk',
 '3zmv',
 '1sqp',
 '5ory',
 '6jad',
 '6t1m',
 '2b1r',
 '1v2j',
 '4m5g',
 '5zma',
 '4rh5',
 '3bh3',
 '6e3n',
 '1g4k',
 '5c5t',
 '5h08',
 

In [25]:
df_plifSpecs

,AA_COORDS,FRAGMENT_ATOMS_COORDS,INTERACTION_TYPE,DIST
FRAGMENT_ID,,,,
70,"[[(28.507, -14.875, 6.05), (28.507, -14.875, 6...","[[(29.075, -14.822, 3.877), (27.378, -16.857, ...",[metal],"[[2.25, 2.28, 2.35, 2.18, 2.50]]"


In [2]:
import os
os.chdir(f'/groups/cherkasvgrp/share/progressive_docking/hmslati/plif_cnn/general_refined_set/3bbb')

In [10]:
import re
content = open("3bbb_ligand.pdb").read()
print(re.findall("^ATOM (.*)", content, re.M))

['     1  O5*   A A   1     -18.744 -22.299  44.582  1.00  0.00           O  ', '     2  C5*   A A   1     -18.829 -22.566  43.185  1.00  0.00           C  ', '     3  C4*   A A   1     -19.139 -21.309  42.388  1.00  0.00           C  ', '     4  O4*   A A   1     -19.503 -21.667  41.032  1.00  0.00           O  ', '     5  C3*   A A   1     -20.292 -20.451  42.917  1.00  0.00           C  ', '     6  O3*   A A   1     -19.790 -19.271  43.570  1.00  0.00           O  ', '     7  C2*   A A   1     -21.131 -20.137  41.673  1.00  0.00           C  ', '     8  C1*   A A   1     -20.246 -20.587  40.511  1.00  0.00           C  ', '     9  N9    A A   1     -20.916 -21.004  39.269  1.00  0.00           N  ', '    10  C8    A A   1     -20.378 -21.817  38.308  1.00  0.00           C  ', '    11  N7    A A   1     -21.161 -22.033  37.277  1.00  0.00           N  ', '    12  C5    A A   1     -22.300 -21.309  37.576  1.00  0.00           C  ', '    13  C6    A A   1     -23.513 -21.126  36.879 